In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 38.0 MB/s 
     |████████████████████████████████| 5.8 MB 64.4 MB/s 
     |████████████████████████████████| 1.3 MB 55.1 MB/s 
     |████████████████████████████████| 452 kB 81.4 MB/s 
     |████████████████████████████████| 7.6 MB 59.0 MB/s 
     |████████████████████████████████| 1.9 MB 63.5 MB/s 
     |████████████████████████████████| 9.2 MB 69.5 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 182 kB 50.9 MB/s 
     |████████████████████████████████| 174 kB 60.6 MB/s 
     |████████████████████████████████| 182 kB 77.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 173 kB 74.2 MB/s 
     |████████████████████████████████| 168 kB 88.2 MB/s 
     |████████████████████████████████| 168 kB 84.7 MB/s 
     |█████████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive')

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d disisbig/hindi-movie-reviews-dataset

  0% 0.00/1.76M [00:00<?, ?B/s]
100% 1.76M/1.76M [00:00<00:00, 79.2MB/s]


In [ ]:
!unzip hindi-movie-reviews-dataset.zip

Archive:  hindi-movie-reviews-dataset.zip
  inflating: train.csv               
  inflating: valid.csv               


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [ ]:
train.head

<bound method NDFrame.head of                                                   text  experience
0    चंद्रमोहन शर्मा को-प्रड्यूसर और लीड ऐक्टर अक्ष...           2
1    अगर आप इस फिल्म को देखने जा रहे हैं तो सबसे पह...           0
2    बॉलीवुड वाले चोरी-छिपे हॉलीवुड फिल्मों से कहान...           2
3    बैनर :\nसंजय दत्त प्रोडक्शन्स प्रा.लि., रुपाली...           0
4    1959 में घटित चर्चित नानावटी कांड में एक क्राइ...           1
..                                                 ...         ...
713  31 अक्टूबर 1984 को काला दिवस कहा जाता है। इस द...           1
714  \n\nगुंडे को देख सत्तर और अस्सी के दशक का सिने...           1
715  Chandermohan.sharma@timesgroup.com ग्लैमर इंडस...           2
716  निर्माता :\nसुनीता गोवारीकर, अजय बिजली, संजीव ...           2
717  फोर्स 2 उन अंडरकवर एजेंट्स को समर्पित है जो सम...           2

[718 rows x 2 columns]>

In [ ]:
print(set(train['experience'].values))

{0, 1, 2}


In [ ]:
train.iloc[:,1]

0      2
1      0
2      2
3      0
4      1
      ..
713    1
714    1
715    2
716    2
717    2
Name: experience, Length: 718, dtype: int64

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
bert = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 5,
    'silent': True
})
bert.train_model(train)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


(450, 1.1042613842752245)

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/valid.csv')

In [ ]:
result, model_outputs, wrong_predictions = bert.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


wrong predictions
118 wrong out of 180


{1: 65, 0: 53}

In [ ]:
display(test)

,text,experience
0,"\n\nबैनर :\nनेक्स्ट जेन फिल्म्स प्रोडक्शन्स, इ...",2
1,इस शुक्रवार रिलीज हुई इकलौती इस हॉलिवुड मूवी क...,2
2,"\n\nबैनर :\nयूटीवी मोशन पिक्चर्स, टिप्स म्युजि...",1
3,"चंद्रमोहन शर्मा, नवभारत टाइम्स पिछले कुछ अर्से...",0
4,\n\nशनाया ग्रोवर (मंजरी फडणीस) पर हत्या का आरो...,0
...,...,...
175,अपनी पिछली फिल्म 'कमांडो' में साउथ फिल्मों में...,0
176,\n\nफिल्म 1971 में सेट है जब ईस्ट पाकिस्तान मे...,2
177,"ए वेडनेस डे, बेबी, स्पेशल 26 जैसी फिल्म बनाने ...",1
178,\nकहानी है मुराद अली मोहम्मद (ऋषि कपूर) की जो ...,2


In [ ]:
model = ClassificationModel('bert', 'monsoon-nlp/hindi-tpu-electra', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 4,
})
model.train_model(train)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/466M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/hindi-tpu-electra were not used when initializing BertForSequenceClassification: ['electra.encoder.layer.11.output.dense.weight', 'electra.encoder.layer.10.output.LayerNorm.bias', 'electra.encoder.layer.2.attention.self.value.weight', 'electra.encoder.layer.9.attention.output.dense.bias', 'electra.encoder.layer.8.attention.output.dense.bias', 'electra.encoder.layer.8.intermediate.dense.weight', 'electra.encoder.layer.5.attention.output.LayerNorm.weight', 'electra.embeddings.word_embeddings.weight', 'electra.encoder.layer.11.output.LayerNorm.weight', 'electra.encoder.layer.2.attention.output.dense.bias', 'electra.encoder.layer.1.attention.self.query.weight', 'electra.encoder.layer.1.output.dense.bias', 'electra.encoder.layer.1.attention.output.LayerNorm.bias', 'electra.encoder.layer.2.attention.self.query.weight', 'electra.encoder.layer.8.output.LayerNorm.weight', 'electra.encoder.layer.7.attention.output.dense.weight', 'electra.encode

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Running Epoch 0 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

(360, 1.124067829383744)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/180 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

wrong predictions
118 wrong out of 180


{1: 65, 0: 53}